# The Reweighting Tutorial
So, you've done some enhanced sampling simulations where you applied a biasing potential to a certain collective variable. For whatever reason, you now want to quantify the free energy surface of the modeled event in a collective variable that is _not the biased CV_. Having applied an artificial bias to the simulations, you cannot simply calculate the probability density of this new CV and convert to free energy. Instead, each frame's weight in the probability density must be measured based on the weight that the frame received in the biased CV space. 

## Analysis Steps:
### 1) Run EMUS or WHAM to get stitching constants in the biased CV FE surfaces
Before any analysis of the new CV can be done, we need to perform a free energy analysis (using either EMUS or WHAM; suggested to use EMUS) of the biased CV space to obtain the "stitching" or "normalization" constant for each window in the baised CV space. This constant is important when reweighting the biased CV sampling into a new collective variable space. 

### 2) Histogram new collective variable data, adding the reweighted counts to the respective histogram bin to calculate the probability density/free energy. 
Each frame's weight is added to a bin based on its respective new CV value. The weight of a frame is calculated using the frame's biased CV value, the stitching constants (calculated from the last step), and the equilibrium constants used for each window of the biased simulations. The corrected weight of a frame from a biased simulation is the inverse of the sum of volume-corrected (if needed) Boltzmann weights from each biasing potential used in the set of analyzed simulations. 

$$ w(t) = \frac{1}{nValues * BinWidth * VolumeCorrection(t) * \frac{1}{nWindows} \sum \limits_{i}^{nWindows} \frac{1}{Z_{i}} exp\left[-\frac{k_{i}}{2k_{B}T}(r(t) - r_{i})^{2}\right]} $$

where $w(t)$ is the weight of a frame, t, $nValues$ is the total number of frames being analyzed, $BinWidth$ is the width of histogram bins used for the new CV histogram, $VolumeCorrection(t)$ is a term included to normalize the volume of the biased CV space and is a frame-dependent value, $nWindows$ is the number of windows being summed over, and the exponential within the sum is the Boltzmann weight of frame t's biased CV value ($r(t)$ relative to window i's equilibrium value, $r_{i}$, and biasing force constant, $k_{i}$) scaled by the window i's stitching constant. The $nValues$ and $BinWidth$ terms are used to convert a count into the associated probability density value. A volume correction term is needed when entropy of states is not equal throughout the biased collective variable space (e.g. more volume at large distances than at close distances). The remaining terms in the denominators represents the average Boltzmann factor associated with the frame's biased CV value within all windows. 

A frame's weight is added to the respective bin in the new CV histogram. 

### 3) Bootstrapping to calculate a lower bound for the error associated with the reweighting results. 
A bootstrapping analysis is used to estimate error for the new reweighted free energy surface. A number of bootstrapping iterations is performed where, during an iteration, $nValues$ of data is randomly pulled from the original collective variable data set. This new dataset is used to calculate the respective, fake free energy surface. The free energy surfaces from all bootstrapping iterations are then used to calculate the standard deviation of the sample data. This error analysis uses random sampling from a dataset with replacement, treating each data point within the original dataset as an independent, uncorrelated data point. Due to this assumption, the standard deviation calculated from bootstrapping represents the lower estimate for error in the reweighted free energy. 

### 4) Perform plotting during each of these steps to function as a sanity check. 
Plotting of probability densities and unstitched free energy graphs are useful to present the quality of sampling and stitching in the new CV space.   

In [ ]:
parameters = {}
IO.config_parser(config_file,parameters)